# Notebook G — 07_simple_classify.ipynb (Rule-Based Classification)

This notebook classifies point clouds using simple rules (Heuristics) without requiring a Machine Learning model.

## Rules used:
- **Ground**: Points with normals pointing up and low local height.
- **Vegetation**: Points that are Green (if color exists) and have noisy normals.
- **Building**: Points with vertical normals (walls) or high flat surfaces (roofs).

In [ ]:
# Cell G0 — Install dependencies
import sys
import subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'numpy', 'open3d', 'scipy', 'laspy'])

In [ ]:
# Cell G1 — Setup & Load Data
from google.colab import drive
import os
import numpy as np
import open3d as o3d
import laspy

drive.mount('/content/drive')

BASE = "/content/drive/MyDrive/voxel_engine"
INPUT_DIR = os.path.join(BASE, "input")
OUTPUT_DIR = os.path.join(BASE, "output", "classified")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Find target file
target_file = None
if os.path.exists(INPUT_DIR):
    files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.ply', '.las', '.laz', '.obj'))]
    if len(files) > 0:
        target_file = files[0]
        print(f"Target file found: {target_file}")
    else:
        print("No files found in input/. Please upload a .ply or .las file.")

# Load
points = None
colors = None

if target_file:
    path = os.path.join(INPUT_DIR, target_file)
    if target_file.lower().endswith('.ply'):
        pcd = o3d.io.read_point_cloud(path)
        points = np.asarray(pcd.points)
        if pcd.has_colors():
            colors = np.asarray(pcd.colors)
        # Estimate normals if missing
        if not pcd.has_normals():
            print("Estimating normals...")
            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
        normals = np.asarray(pcd.normals)
        
    print(f"Loaded {points.shape[0]} points.")

In [ ]:
# Cell G2 — Simple Classification Logic

def classify_heuristic(pts, nrms, cols):
    labels = np.zeros(pts.shape[0], dtype=np.uint8)
    # 0=Unclassified, 1=Ground, 2=Vegetation, 3=Building/Structure

    # --- 1. Ground Detection (Simple Normal check) ---
    # Points with normal Z close to 1 (pointing up)
    # And absolute Z is relatively low (this is naive, usually relies on cloth simulation)
    upward_normals = nrms[:, 2] > 0.9
    ground_mask = upward_normals
    labels[ground_mask] = 1

    # --- 2. Vegetation Detection (Green Color check) ---
    if cols is not None:
        # Assuming RGB in 0-1
        # Green dominance: G > R and G > B
        is_green = (cols[:, 1] > cols[:, 0]) & (cols[:, 1] > cols[:, 2])
        # Also check for somewhat rough normals (not perfectly flat)
        is_rough = nrms[:, 2] < 0.95
        
        veg_mask = is_green & is_rough & (labels == 0)
        labels[veg_mask] = 2

    # --- 3. Structure/Building (Vertical surfaces) ---
    # Normals that are horizontal (z near 0)
    is_wall = (np.abs(nrms[:, 2]) < 0.2)
    struct_mask = is_wall & (labels == 0)
    labels[struct_mask] = 3
    
    return labels

if points is not None:
    print("Classifying...")
    labels = classify_heuristic(points, normals, colors)
    
    # Statistics
    classes, counts = np.unique(labels, return_counts=True)
    print("Results:", dict(zip(classes, counts)))
    # Key: 1=Ground, 2=Vegetation, 3=Building

In [ ]:
# Cell G3 — Visualize & Save
import matplotlib.pyplot as plt

# Create colored point cloud based on labels
label_colors = np.zeros_like(points)
# 1=Ground (Brown), 2=Veg (Green), 3=Build (Red), 0=Other (Grey)
label_colors[labels == 0] = [0.5, 0.5, 0.5]
label_colors[labels == 1] = [0.6, 0.5, 0.3]
label_colors[labels == 2] = [0.0, 0.8, 0.0]
label_colors[labels == 3] = [1.0, 0.0, 0.0]

out_pcd = o3d.geometry.PointCloud()
out_pcd.points = o3d.utility.Vector3dVector(points)
out_pcd.colors = o3d.utility.Vector3dVector(label_colors)
out_pcd.normals = o3d.utility.Vector3dVector(normals)

out_name = "classified_heuristic.ply"
o3d.io.write_point_cloud(os.path.join(OUTPUT_DIR, out_name), out_pcd)
print(f"Saved classified pointcloud to {os.path.join(OUTPUT_DIR, out_name)}")

# Save as NumPy for next pipeline steps
np.save(os.path.join(OUTPUT_DIR, "classified_points.npy"), points.astype(np.float32))
np.save(os.path.join(OUTPUT_DIR, "classified_labels.npy"), labels.astype(np.uint8))
print("Saved .npy files for pipeline usage.")